In [19]:
import numpy as np 
import pandas as pd  

In [20]:
data = {
    "Activity": list(range(1, 28)),
    "Activity Time (mins)": [6, 10, 7, 12, 18, 8, 19, 6, 18, 15, 14, 19, 15, 9, 12, 5, 12, 11, 8, 12, 8, 17, 12, 8, 7, 12, 11],
    "Preceding Activities": [
        None, None, None, None, None, "1,3", "2,4", "3,5", "2", "8,9", "6,10", "7,8,11", "11,12", "13", "14", "6,11,14",
        "16", "16", "16", "17,19", "17,18", "20,21", "19,20", "14,22", "13,15", "3,4", "25,26"
    ]
}

In [21]:
df = pd.DataFrame(data)

In [22]:
df["Preceding Activities"] = df["Preceding Activities"].apply(lambda x: [] if pd.isna(x) else [int(i) for i in str(x).split(",")])

df.head()

,Activity,Activity Time (mins),Preceding Activities
0,1,6,[]
1,2,10,[]
2,3,7,[]
3,4,12,[]
4,5,18,[]


# Q1 
We need to determine the maximum time assigned to any workstation after optimally distributing activities 

**Solution Approach**
we need to find the workstation with the maximum total assigned time

In [23]:
cycle_time = max(df["Activity Time (mins)"])

def assign_workstations(df, cycle_time):
    workstations = []
    remaining_tasks = df.copy()
    
    while not remaining_tasks.empty:
        workstation = []
        total_time = 0
        
        for i, row in remaining_tasks.iterrows():
            if total_time + row["Activity Time (mins)"] <= cycle_time:
                workstation.append(row["Activity"])
                total_time += row["Activity Time (mins)"]
        

        workstations.append((workstation, total_time))
        
 
        remaining_tasks = remaining_tasks[~remaining_tasks["Activity"].isin(workstations[-1][0])]

    
    return workstations

workstations = assign_workstations(df, cycle_time)


max_activity_time = max([wt[1] for wt in workstations])

print(f"Maximum activity time among workstations: {max_activity_time} minutes")


Maximum activity time among workstations: 19 minutes


# Q2 What is the maximum cycle time in minutes ?


In [24]:
max_cycle_time = max(df["Activity Time (mins)"])

print(f"Maximum cycle time: {max_cycle_time} minutes")


Maximum cycle time: 19 minutes


# Q3 What is the maximum possible output that can be achieved for a week? 


$Maximum Output = \frac{Total Available Time per Week}{Cycle Time}$

In [25]:
shifts_per_day = 2
hours_per_shift = 8
minutes_per_hour = 60
days_per_week = 5

total_available_time = shifts_per_day * hours_per_shift * minutes_per_hour * days_per_week

maximum_output_per_week = total_available_time // cycle_time

print(f"Maximum possible output per week: {maximum_output_per_week} units")


Maximum possible output per week: 252 units


# Q4 What is the theoretical minimum number of workstations that are required to assign the 50 activities?

$Theoretical Minimum Workstation = \frac{Total Activvity Time}{Cycle Time}$

In [26]:
total_activity_time = df["Activity Time (mins)"].sum()

theoretical_min_workstations = np.ceil(total_activity_time / cycle_time)

print(f"Theoretical Minimum Workstations: {int(theoretical_min_workstations)}")


Theoretical Minimum Workstations: 17


# Q5 How many workstations are actually required as per the optimized assignment?

In [27]:
def assign_workstations(df, cycle_time):
    remaining_tasks = df.copy()
    workstations = []

    while not remaining_tasks.empty:
        workstation = []
        total_time = 0
        available_tasks = remaining_tasks[remaining_tasks["Preceding Activities"].apply(lambda x: all(i not in remaining_tasks["Activity"].values for i in x))]

        for _, row in available_tasks.iterrows():
            if total_time + row["Activity Time (mins)"] <= cycle_time:
                workstation.append(row["Activity"])
                total_time += row["Activity Time (mins)"]

        workstations.append(workstation)
        remaining_tasks = remaining_tasks[~remaining_tasks["Activity"].isin(workstation)]

    return workstations


workstations = assign_workstations(df, cycle_time)


actual_workstations = len(workstations)
print(f"Actual Workstations Required: {actual_workstations}")


Actual Workstations Required: 21


# Q6 What is the efficiency of the optimized layout?

$Efficiency= \frac{Total Task Time}{Number of Workstation \times Cycle Time}\times100$ 
 

In [29]:
total_task_time = df["Activity Time (mins)"].sum()

efficiency = (total_task_time / (actual_workstations * cycle_time)) * 100


efficiency = round(efficiency, 2)

print(f"Efficiency of the optimized layout: {efficiency}%")


Efficiency of the optimized layout: 77.94%


# Q7 What is the minimum efficiency of a workstation in the optimized layout?


$Workstation Efficiency= \frac{Total Activity Time in Workstation}{Cycle Time}\times100$ 

In [33]:
# print(workstations)


workstation_efficiencies = []

for ws in workstations:
    total_time = sum(df.loc[df["Activity"].isin(ws), "Activity Time (mins)"])
    efficiency = (total_time / cycle_time) * 100
    workstation_efficiencies.append(efficiency)

min_efficiency = round(min(workstation_efficiencies), 2)

print(f"Minimum efficiency of a workstation in the optimized layout: {min_efficiency}%")


Minimum efficiency of a workstation in the optimized layout: 42.11%


# Q8 How many workstations have the maximum efficiency that is possible for a workstation in the optimized layout (choose all that is applicable)?


In [34]:
workstation_efficiencies = []

for ws in workstations:
    total_time = sum(df.loc[df["Activity"].isin(ws), "Activity Time (mins)"])
    efficiency = (total_time / cycle_time) * 100
    workstation_efficiencies.append(efficiency)


In [35]:
max_efficiency = max(workstation_efficiencies)


In [36]:
max_eff_count = workstation_efficiencies.count(max_efficiency)

print(f"Number of workstations with maximum efficiency: {max_eff_count}")


Number of workstations with maximum efficiency: 5


# Q9 How many workstations have 2 or more assigned activities in the optimized layout? 

In [38]:
workstations_with_2_or_more = sum(1 for ws in workstations if len(ws) >= 2)

print(f"Number of workstations with 2 or more assigned activities: {workstations_with_2_or_more}")


Number of workstations with 2 or more assigned activities: 6


# Q10 How many workstations have only one assigned activity in the optimized layout? 


In [39]:
workstations_with_1_activity = sum(1 for ws in workstations if len(ws) == 1)

print(f"Number of workstations with only one assigned activity: {workstations_with_1_activity}")


Number of workstations with only one assigned activity: 15
